In [12]:
import azureml.core, cv2, keras, io, json, ntpath, os, requests, urllib
print("SDK version:", azureml.core.VERSION)

import matplotlib.pyplot as plt
import numpy as np

from azureml.core import Datastore, Experiment, ScriptRunConfig, Workspace
from azureml.core.compute import AmlCompute, ComputeTarget
from azureml.core.compute_target import ComputeTargetException
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core.dataset import Dataset
from azureml.core.runconfig import DEFAULT_CPU_IMAGE, RunConfiguration
from azureml.data.data_reference import DataReference
from azureml.pipeline.core import Pipeline, PipelineData
from azureml.pipeline.steps import PythonScriptStep
from azureml.train.estimator import Estimator
from azureml.widgets import RunDetails
from pathlib import Path
from PIL import Image
from sklearn.model_selection import train_test_split

SDK version: 1.0.57


In [13]:
%matplotlib inline

In [14]:
ws = Workspace.from_config()
print('Name: {0}'.format(ws.name), 'Resource Group: {0}'.format(ws.resource_group), 'Location: {0}'.format(ws.location), 'Subscription Id: {0}'.format(ws.subscription_id), sep = '\n')

Name: HealthyHabitatAI
Resource Group: HealthyHabitatAI
Location: australiaeast
Subscription Id: 3191ba83-be2b-4b29-8409-f06e2fbb65bd


In [15]:
datastore = Datastore.register_azure_blob_container(workspace=ws, 
                                                    datastore_name='raw', 
                                                    container_name='raw',
                                                    account_name='healthyhabitatai',
                                                    account_key='rjED2w2OBOH5fRw7mn7w4pCx20Qgnsoqv3L4fOYIygml+fwWHeUUys/5rnorT2mtTTU5pudAQf2yTXaxwGjN/Q==',
                                                    create_if_not_exists=False)

In [16]:
compute_name = 'GPU'

if compute_name in ws.compute_targets:
    compute_target = ws.compute_targets[compute_name]
    
    if compute_target and type(compute_target) is AmlCompute:
        print('Found compute target: ' + compute_name)
else:
    provisioning_configuration = AmlCompute.provisioning_configuration(vm_size = 'STANDARD_NC6',
                                                                       min_nodes = 1,
                                                                       max_nodes = 1)

    compute_target = ComputeTarget.create(ws, compute_name, provisioning_configuration)
    
    compute_target.wait_for_completion(show_output=True, min_node_count=None, timeout_in_minutes=20)
    
    print(compute_target.status.serialize())

Creating
Succeeded............
AmlCompute wait for completion finished
Minimum number of nodes requested have been provisioned
{'currentNodeCount': 1, 'targetNodeCount': 1, 'nodeStateCounts': {'preparingNodeCount': 1, 'runningNodeCount': 0, 'idleNodeCount': 0, 'unusableNodeCount': 0, 'leavingNodeCount': 0, 'preemptedNodeCount': 0}, 'allocationState': 'Steady', 'allocationStateTransitionTime': '2019-10-24T09:32:09.450000+00:00', 'errors': None, 'creationTime': '2019-10-24T09:30:48.088492+00:00', 'modifiedTime': '2019-10-24T09:31:07.043265+00:00', 'provisioningState': 'Succeeded', 'provisioningStateTransitionTime': None, 'scaleSettings': {'minNodeCount': 1, 'maxNodeCount': 1, 'nodeIdleTimeBeforeScaleDown': ''}, 'vmPriority': 'Dedicated', 'vmSize': 'STANDARD_NC6'}


In [19]:
compute_name = 'CPU'

if compute_name in ws.compute_targets:
    compute_target = ws.compute_targets[compute_name]
    
    if compute_target and type(compute_target) is AmlCompute:
        print('Found compute target: ' + compute_name)
else:
    provisioning_configuration = AmlCompute.provisioning_configuration(vm_size = 'STANDARD_D2_V2',
                                                                       min_nodes = 1,
                                                                       max_nodes = 1)

    compute_target = ComputeTarget.create(ws, compute_name, provisioning_configuration)
    
    compute_target.wait_for_completion(show_output=True, min_node_count=None, timeout_in_minutes=20)
    
    print(compute_target.status.serialize())

Creating
Succeeded...........
AmlCompute wait for completion finished
Minimum number of nodes requested have been provisioned
{'currentNodeCount': 1, 'targetNodeCount': 1, 'nodeStateCounts': {'preparingNodeCount': 1, 'runningNodeCount': 0, 'idleNodeCount': 0, 'unusableNodeCount': 0, 'leavingNodeCount': 0, 'preemptedNodeCount': 0}, 'allocationState': 'Steady', 'allocationStateTransitionTime': '2019-10-24T09:40:04.356000+00:00', 'errors': None, 'creationTime': '2019-10-24T09:38:54.252537+00:00', 'modifiedTime': '2019-10-24T09:39:10.998736+00:00', 'provisioningState': 'Succeeded', 'provisioningStateTransitionTime': None, 'scaleSettings': {'minNodeCount': 1, 'maxNodeCount': 1, 'nodeIdleTimeBeforeScaleDown': ''}, 'vmPriority': 'Dedicated', 'vmSize': 'STANDARD_D2_V2'}


In [ ]:
input_data = DataReference(
    datastore=Datastore(ws, blob_store),
    data_reference_name="test_data",
    path_on_datastore="20newsgroups/20news.pkl")

In [20]:
DEFAULT_CPU_IMAGE

'mcr.microsoft.com/azureml/base:intelmpi2018.3-ubuntu16.04'

In [21]:
source_directory = './pytorch-semseg'

conda_dependencies = CondaDependencies()
conda_dependencies.add_pip_package('numpy')
conda_dependencies.add_pip_package('pyyaml')
conda_dependencies.add_pip_package('scipy')
conda_dependencies.add_pip_package('tensorboardx')
conda_dependencies.add_pip_package('torch')
conda_dependencies.add_pip_package('torchvision')
conda_dependencies.add_pip_package('tqdm')
conda_dependencies.add_pip_package('matplotlib')

run_configuration = RunConfiguration()
run_configuration.environment.docker.enabled = True
run_configuration.environment.docker.base_image = DEFAULT_CPU_IMAGE
run_configuration.environment.python.user_managed_dependencies = False
run_configuration.environment.python.conda_dependencies = conda_dependencies

train_step = PythonScriptStep(name="train_step",
                              script_name="train.py",
                              arguments=['--config', './configs/unet_cityscapes.yml'],
                              compute_target=compute_target,
                              source_directory=source_directory,
                              runconfig=run_configuration,
                              allow_reuse=False)

In [22]:
pipeline = Pipeline(workspace=ws, steps=[train_step])

In [23]:
pipeline_run = Experiment(ws, 'train_step').submit(pipeline)
pipeline_run

Created step train_step [100324e1][d10493d2-45f8-499d-b44b-6ad8042bf039], (This step will run and generate new outputs)
Submitted pipeline run: 05780bc1-3be0-4a33-b257-c042848fe22c


Experiment,Id,Type,Status,Details Page,Docs Page
train_step,05780bc1-3be0-4a33-b257-c042848fe22c,azureml.PipelineRun,Running,Link to Azure Portal,Link to Documentation


In [24]:
RunDetails(pipeline_run).show()

_PipelineWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', …

In [25]:
pipeline_run.wait_for_completion(show_output=True)

PipelineRunId: 05780bc1-3be0-4a33-b257-c042848fe22c
Link to Portal: https://mlworkspace.azure.ai/portal/subscriptions/3191ba83-be2b-4b29-8409-f06e2fbb65bd/resourceGroups/HealthyHabitatAI/providers/Microsoft.MachineLearningServices/workspaces/HealthyHabitatAI/experiments/train_step/runs/05780bc1-3be0-4a33-b257-c042848fe22c
PipelineRun Status: Running


StepRunId: 231d0aba-2b05-4028-a095-e58c7e53c969
Link to Portal: https://mlworkspace.azure.ai/portal/subscriptions/3191ba83-be2b-4b29-8409-f06e2fbb65bd/resourceGroups/HealthyHabitatAI/providers/Microsoft.MachineLearningServices/workspaces/HealthyHabitatAI/experiments/train_step/runs/231d0aba-2b05-4028-a095-e58c7e53c969
StepRun( train_step ) Status: Running

Streaming azureml-logs/20_image_build_log.txt
2019/10/24 09:44:08 Downloading source code...
2019/10/24 09:44:10 Finished downloading source code
2019/10/24 09:44:10 Creating Docker network: acb_default_network, driver: 'bridge'
2019/10/24 09:44:11 Successfully set up Docker network: ac

Executing transaction: ...working... done


  Created wheel for pyyaml: filename=PyYAML-5.1.2-cp36-cp36m-linux_x86_64.whl size=44104 sha256=22cfadaa9a0d035abae611d3086ba2f79fc966e62b2fade1887836afee5224df
  Stored in directory: /root/.cache/pip/wheels/d9/45/dd/65f0b38450c47cf7e5312883deb97d065e030c5cca0a365030
  Created wheel for json-logging-py: filename=json_logging_py-0.2-cp36-none-any.whl size=3924 sha256=6438c517580c21b0ec2861df7c75f033e342655897b7e405d19f223091120376
  Stored in directory: /root/.cache/pip/wheels/0d/2e/1c/c638b7589610d8b9358a6e5eb008edacb8b3e9b6d1edc9479f
  Created wheel for pathspec: filename=pathspec-0.6.0-cp36-none-any.whl size=26671 sha256=b385cb4dc716a31149988e25536e51afe37be78602b43c9b54bfc6c56251205e
  Stored in directory: /root/.cache/pip/wheels/62/b8/e1/e2719465b5947c40cd85d613d6cb33449b86a1ca5a6c574269
  Created wheel for dill: filename=dill-0.3.1.1-cp36-none-any.whl size=78532 sha256=9fec0dce4deab1574add739f7ffd2fddca525ff99d503a3acf11a114509dd625
  Stored in directory: /root/.cache/pip/wheels/5

 ---> Running in a96a04e30d48
Removing intermediate container a96a04e30d48
 ---> aa7c12fbba54
Step 11/15 : ENV LD_LIBRARY_PATH /azureml-envs/azureml_b4ae91ae46af9a2383b5f8d921161272/lib:$LD_LIBRARY_PATH
 ---> Running in 9e993fbf67aa
Removing intermediate container 9e993fbf67aa
 ---> c85fc46116e7
Step 12/15 : COPY azureml-environment-setup/spark_cache.py azureml-environment-setup/log4j.properties /azureml-environment-setup/
 ---> 4a7bf9d9faf9
Step 13/15 : RUN if [ $SPARK_HOME ]; then /bin/bash -c '$SPARK_HOME/bin/spark-submit "--repositories" "[]" /azureml-environment-setup/spark_cache.py'; fi
 ---> Running in d443404da5fb
Removing intermediate container d443404da5fb
 ---> e0f23f5b4df2
Step 14/15 : ENV AZUREML_ENVIRONMENT_IMAGE True
 ---> Running in 77ea0fdfc706
Removing intermediate container 77ea0fdfc706
 ---> c78ca589a10c
Step 15/15 : CMD ["bash"]
 ---> Running in dfa748636ce2
Removing intermediate container dfa748636ce2
 ---> 42699d803fdc
Successfully built 42699d803fdc
Successfully

WARNING - Retrying (Retry(total=2, connect=3, read=2, redirect=None, status=None)) after connection broken by 'ProtocolError('Connection aborted.', OSError("(10054, 'WSAECONNRESET')",))': /azureml/ExperimentRun/dcid.231d0aba-2b05-4028-a095-e58c7e53c969/azureml-logs/75_job_post-tvmps_0b2daac0cde1749b2b1b89d01b386e6883ea4b9d4ca79fbf8fcb2a0ef85d3791_d.txt?sv=2018-11-09&sr=b&sig=O%2BW%2FOex6CWxWzIVArt5pkwEWFhDM0pmDwO4GBUFSQ%2B8%3D&st=2019-10-24T09%3A54%3A18Z&se=2019-10-24T18%3A04%3A18Z&sp=r



Streaming azureml-logs/75_job_post-tvmps_0b2daac0cde1749b2b1b89d01b386e6883ea4b9d4ca79fbf8fcb2a0ef85d3791_d.txt
bash: /azureml-envs/azureml_b4ae91ae46af9a2383b5f8d921161272/lib/libtinfo.so.5: no version information available (required by bash)
Starting job release. Current time:2019-10-24T09:53:52.611386
Logging experiment finalizing status in history service.
Starting the daemon thread to refresh tokens in background for process with pid = 179
Job release is complete. Current time:2019-10-24T09:53:54.529911

StepRun(train_step) Execution Summary
StepRun( train_step ) Status: Failed

Warnings:
{
  "error": {
    "code": "UserError",
    "message": "AzureMLCompute job failed.\nJobFailed: Submitted script failed with a non-zero exit code; see the driver log file for details.",
    "detailsUri": null,
    "target": null,
    "details": [],
    "innerError": null,
    "debugInfo": null
  },
  "correlation": {
    "operation": null,
    "request": "Gj6VcxGMcm8="
  },
  "environment": "aust

ActivityFailedException: ActivityFailedException:
	Message: Activity Failed:
{
    "error": {
        "code": "UserError",
        "message": "User program failed with FileNotFoundError: [Errno 2] No such file or directory: 'configs/unet_cityscapes.yml'",
        "details": [],
        "debugInfo": {
            "type": "FileNotFoundError",
            "message": "[Errno 2] No such file or directory: 'configs/unet_cityscapes.yml'",
            "stackTrace": "  File \"/mnt/batch/tasks/shared/LS_root/jobs/healthyhabitatai/azureml/231d0aba-2b05-4028-a095-e58c7e53c969/mounts/workspaceblobstore/azureml/231d0aba-2b05-4028-a095-e58c7e53c969/azureml-setup/context_manager_injector.py\", line 99, in execute_with_context\n    runpy.run_path(sys.argv[0], globals(), run_name=\"__main__\")\n  File \"/azureml-envs/azureml_b4ae91ae46af9a2383b5f8d921161272/lib/python3.6/runpy.py\", line 263, in run_path\n    pkg_name=pkg_name, script_name=fname)\n  File \"/azureml-envs/azureml_b4ae91ae46af9a2383b5f8d92

In [ ]:
children = pipeline_run.get_children()
for child in children:
    status = child.get_status()
    print('Id:', child.id, 'Script:', child.name, 'Status:', status)
    RunDetails(child).show()